In [12]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [13]:
rent_obj = []
all_property = []

for page in range(19,29):
    link = 'https://www.apartments.com/Townhomes/ca/' + str(page) + '/'
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='basic placard')

    # Loop through returned results and get the hyperlink of the properties of the page
    for result in results:
        find_tag = result.find('a')['href']
        new_response = requests.get(find_tag, headers=headers)
        new_soup = BeautifulSoup(new_response.text, 'html.parser')
        new_results = new_soup.find('div', class_="mainWrapper")
        
        full_address = result.find('div', class_ = "location").text
        city = new_results.find('div', class_ = "propertyAddress").select('span')[0].text
        state = new_results.find('div', class_ = "propertyAddress").select('span')[1].text
        zip_code = new_results.find('div', class_ = "propertyAddress").select('span')[2].text
        street = new_results.find('h1', class_ = "propertyName").text.replace('\r\n                    ', "").strip()
        bedroom_container = new_results.find('div', class_="rentRollupContainer")
        house_details = new_results.find('table', class_="availabilityTable")
        bath_number = house_details.find('td', class_="baths").find('span', class_="shortText").text.replace('\r\n                    ', "").strip()
        house_square_feet = house_details.find('td', class_="sqft").text
        square_feet = house_square_feet.split(" ")[0];
        lease_length = house_details.find('td', class_="leaseLength").text
        availability_check = house_details.find('td', class_="available").text.replace('\r\n                    ', "").strip()
        all_the_rents = bedroom_container.findAll('span', class_="rentRollup")

        bedroom_info = []
        for rent_block in all_the_rents:
            bedroom_type = rent_block.find('span', class_="shortText").text
            try:
                price = re.search('\$.* ', rent_block.text)
                price = price.group(0)[:-3]

            except:
                if "Call for Rent" in rent_block.text:
                    price = ""
                else:
                    price = re.search('\$.*', rent_block.text).group()[:-1]
            bedroom_info.append({
                    'bedroom_type': bedroom_type,
                    'price': price
                })
        rent_obj = {
                "City_Rent": city,
                "Type": "Townhomes",
                "State_Rent": state,
                "Zip_code": zip_code,
                "Bedroom_Info": bedroom_info,
                "Full_Address": full_address,
                "Rent_link": find_tag,
                "Bath_Info": bath_number,
                "Square_Feet_Data": house_square_feet,
                "Square_Feet": square_feet,
                "Lease_length": lease_length,
                "Availability_check": availability_check
            }
        for room_type in bedroom_info:
            rent_obj[room_type["bedroom_type"]] = room_type["price"];
        
        all_property.append(rent_obj)
        time.sleep(2)

In [14]:
rent_data = pd.DataFrame(all_property)
rent_data

,1 Bed,2 Beds,3 Beds,4 Beds,Availability_check,Bath_Info,Bedroom_Info,City_Rent,Full_Address,Lease_length,Rent_link,Square_Feet,Square_Feet_Data,State_Rent,Studio,Type,Zip_code
0,NaN,NaN,"$4,200",NaN,Available Now,2½ BAs,"[{'bedroom_type': '3 Beds', 'price': '$4,200'}]",3279 Mt Diablo Ct,"Lafayette, CA 94549",,https://www.apartments.com/3279-mt-diablo-ct-l...,"1,731","1,731 Sq Ft",Lafayette,NaN,Townhomes,CA
1,NaN,"$2,095",NaN,NaN,Available Now,1½ BAs,"[{'bedroom_type': '2 Beds', 'price': '$2,095'}]",9 S Lake Dr,"Antioch, CA 94509",,https://www.apartments.com/9-s-lake-dr-antioch...,"3,900","3,900 Sq Ft",Antioch,NaN,Townhomes,CA
2,NaN,NaN,"$5,450",NaN,Available Now,2 BAs,"[{'bedroom_type': '3 Beds', 'price': '$5,450'}]",6401 Commodore Sloat Dr,"Los Angeles, CA 90048",,https://www.apartments.com/6401-commodore-sloa...,,,Los Angeles,NaN,Townhomes,CA
3,NaN,NaN,"$3,800",NaN,Available Now,3 BAs,"[{'bedroom_type': '3 Beds', 'price': '$3,800'}]",222 S Marengo Ave,"Pasadena, CA 91101",,https://www.apartments.com/222-s-marengo-ave-p...,,,Pasadena,NaN,Townhomes,CA
4,NaN,NaN,"$3,900",NaN,Available Now,3 BAs,"[{'bedroom_type': '3 Beds', 'price': '$3,900'}]",148 Tivoli Ln,"Danville, CA 94506",,https://www.apartments.com/148-tivoli-ln-danvi...,"1,939","1,939 Sq Ft",Danville,NaN,Townhomes,CA
5,NaN,"$1,695",NaN,NaN,Available Now,1½ BAs,"[{'bedroom_type': '2 Beds', 'price': '$1,695'}]",4331 53rd St,"San Diego, CA 92115",,https://www.apartments.com/4331-53rd-st-san-di...,950,950 Sq Ft,San Diego,NaN,Townhomes,CA
6,NaN,"$3,300",NaN,NaN,Available Now,2½ BAs,"[{'bedroom_type': '2 Beds', 'price': '$3,300'}]",305 Windstream Pl,"Danville, CA 94526",,https://www.apartments.com/305-windstream-pl-d...,"1,764","1,764 Sq Ft",Danville,NaN,Townhomes,CA
7,"$4,250",NaN,NaN,NaN,Available Now,1 BA,"[{'bedroom_type': '1 Bed', 'price': '$4,250'}]",22605 Pacific Coast Hwy,"Malibu, CA 90265",,https://www.apartments.com/22605-pacific-coast...,,,Malibu,NaN,Townhomes,CA
8,NaN,NaN,"$2,900",NaN,Available Now,1 BA,"[{'bedroom_type': '3 Beds', 'price': '$2,900'}]",4286 S Hobart Blvd,"Los Angeles, CA 90062",,https://www.apartments.com/4286-s-hobart-blvd-...,,,Los Angeles,NaN,Townhomes,CA
9,NaN,"$8,000",NaN,NaN,Available Now,3 BAs,"[{'bedroom_type': '2 Beds', 'price': '$8,000'}]",185 S Rodeo Dr,"Beverly Hills, CA 90212",,https://www.apartments.com/185-s-rodeo-dr-beve...,,,Beverly Hills,NaN,Townhomes,CA


In [15]:
# Export file as a CSV, without the Pandas index, but with the header
rent_data.to_csv("Output/Apartment.com/Q4-2019_Apartment.com_Townhomes_Data_page19-28.csv", index=False, header=True)